In [1]:
!pip install numpy
!pip install pandas
!pip install tensorflow
!pip install scikit-learn
!pip install matplotlib
!pip install gensim


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.neighbors import NearestNeighbors

In [3]:
#reading the data
data = pd.read_csv('all_datav3.csv', delimiter=';', on_bad_lines='skip')
display(data)

,Kunde,customname,customform,name,fieldobjectid,fieldparentid,fieldparenttype,elementtype,fieldtype,blocktype,specialtype,fieldrelation,fieldlabel,language
0,abinventech,form876,876,Kundereklamation,877,878.0,customformelement,field,listselect,NaN,NaN,form2488,Vælg kunde,DA
1,abinventech,form876,876,Kundereklamation,878,NaN,customform,block,NaN,layout_fieldset,NaN,NaN,Kundeoplysninger,DA
2,abinventech,form876,876,Kundereklamation,879,878.0,customformelement,field,string,NaN,NaN,NaN,Gadenavn,DA
3,abinventech,form876,876,Kundereklamation,880,878.0,customformelement,field,string,NaN,NaN,NaN,Postnr,DA
4,abinventech,form876,876,Kundereklamation,881,878.0,customformelement,field,string,NaN,NaN,NaN,By,DA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9446,ipw,form1683675,1683675,SI - Handleplan,1711069,2359135.0,customformelement,field,relation,NaN,NaN,form1683707,Evt. anden strategisk initiativ,DA
9447,ipw,form1683793,1683793,SI - Aktiviteter,1711096,NaN,customform,field,texthistory,NaN,NaN,NaN,Fremdrift,DA
9448,ipw,form1683793,1683793,SI - Aktiviteter,1711098,NaN,customform,field,autonum,NaN,NaN,NaN,ID,DA
9449,ipw,form1683675,1683675,SI - Handleplan,1711129,1729633.0,customformelement,field,autonum,NaN,NaN,NaN,ID,DA


# Data Processing


## Field Type Handling

1. Ensure that the correct field type values are taken from the data source (special).
2. Write these values into their corresponding `fieldtype` columns.

## Grouping Forms

- Group the forms based on their **ID** and **Name**.
- This allows for better organization and easier access to related data.

## Mapping Field Types and Labels

- After grouping, map each **fieldtype** with its corresponding **field label**.
- This mapping ensures that each field is appropriately identified and categorized for further use.


In [4]:
# Remove all elements of type block.(maybe late include it)
indices_to_drop = data[data['elementtype'] == 'block'].index

# Drop those indices
data = data.drop(indices_to_drop)

# If the element is special the actual fieldtype is in specialtype

data['isspecial'] = data['elementtype'].apply(lambda x: 1 if x == 'special' else 0)

for index, row in data.iterrows():
    if row['isspecial'] == 1:
        if pd.notnull(row['fieldtype']):  # Check if 'fieldtype' is already filled
            print(f"Row {index}: 'fieldtype' already has a value before assigning 'specialtype'")
        else:
            # If 'fieldtype' is empty, assign the value from 'specialtype'
            data.at[index, 'fieldtype'] = row['specialtype']

#Drop non-relevant columns.
data = data.drop(columns=['Kunde','isspecial','elementtype', 'customname','fieldobjectid','fieldparentid','fieldparenttype','blocktype','fieldrelation','language','specialtype'])

#Lets group the data by customform (id) and include name and mape the structure.
result = data.groupby(['customform', 'name']).agg({
    'fieldtype': lambda x: list(x),  # Keep all field types as a list
    'fieldlabel': lambda x: list(x)  # keep all field labels as a list
}).reset_index()

display(result)

,customform,name,fieldtype,fieldlabel
0,770,Projektportefølje,"[autonum, created, createdby, relation, string...","[Projekt nr., Oprettet den, Oprettet af, Proje..."
1,775,Projektområde,[string],[Tekst]
2,876,Claims,"[userrelation, text, relation, text, createdby...","[Processor, Description of claim, Complaint ca..."
3,876,Kundereklamation,"[listselect, string, string, string, string, s...","[Vælg kunde, Gadenavn, Postnr, By, Telefonnumm..."
4,876,Reklamation,"[listselect, string, string, userrelation, rel...","[Vælg kunde, E-mail, Kontaktperson, Behandles ..."
...,...,...,...,...
753,1683707,Strategiske initiativer,"[text, relation, string, relation]","[Tekst, MWB, Id, Periode]"
754,1683712,SI - Must win battles,[string],[MWB]
755,1683736,SI - perioder,[string],[Periode]
756,1683793,SI - Aktiviteter,"[text, date, combo, userrelation, relation, te...","[Aktivitet, Deadline, Status, Superviserende l..."


In [5]:
dfVal = pd.read_csv('all_data-validationCSV.csv', delimiter=';', on_bad_lines='skip')
# Remove all elements of type block.(maybe late include it)
indices_to_drop = dfVal[dfVal['elementtype'] == 'block'].index

# Drop those indices
dfVal = dfVal.drop(indices_to_drop)

# If the element is special the actual fieldtype is in specialtype

dfVal['isspecial'] = dfVal['elementtype'].apply(lambda x: 1 if x == 'special' else 0)

for index, row in dfVal.iterrows():
    if row['isspecial'] == 1:
        if pd.notnull(row['fieldtype']):  # Check if 'fieldtype' is already filled
            print(f"Row {index}: 'fieldtype' already has a value before assigning 'specialtype'")
        else:
            # If 'fieldtype' is empty, assign the value from 'specialtype'
            dfVal.at[index, 'fieldtype'] = row['specialtype']

#Drop non-relevant columns.
dfVal = dfVal.drop(columns=['Kunde','isspecial','elementtype', 'customname','fieldobjectid','fieldparentid','fieldparenttype','blocktype','fieldrelation','language','specialtype'])

#Lets group the data by customform (id) and include name and mape the structure.
dataVal = dfVal.groupby(['customform', 'name']).agg({
    'fieldtype': lambda x: list(x),  # Keep all field types as a list
    'fieldlabel': lambda x: list(x)  # keep all field labels as a list
}).reset_index()

display(result)

,customform,name,fieldtype,fieldlabel
0,770,Projektportefølje,"[autonum, created, createdby, relation, string...","[Projekt nr., Oprettet den, Oprettet af, Proje..."
1,775,Projektområde,[string],[Tekst]
2,876,Claims,"[userrelation, text, relation, text, createdby...","[Processor, Description of claim, Complaint ca..."
3,876,Kundereklamation,"[listselect, string, string, string, string, s...","[Vælg kunde, Gadenavn, Postnr, By, Telefonnumm..."
4,876,Reklamation,"[listselect, string, string, userrelation, rel...","[Vælg kunde, E-mail, Kontaktperson, Behandles ..."
...,...,...,...,...
753,1683707,Strategiske initiativer,"[text, relation, string, relation]","[Tekst, MWB, Id, Periode]"
754,1683712,SI - Must win battles,[string],[MWB]
755,1683736,SI - perioder,[string],[Periode]
756,1683793,SI - Aktiviteter,"[text, date, combo, userrelation, relation, te...","[Aktivitet, Deadline, Status, Superviserende l..."


# Embeddings

In this section, we focus on converting form data into sentence embeddings and storing them.

## Converting Form to Sentence

1. Convert each form into a sentence that includes the following details:
   - **Form Name**
   - **Field Type**
   - **Field Label**
## Creating Embeddings

2. After generating the sentence, create an embedding for each one. This embedding represents the form as a vector for easier comparison and analysis.

3. Store these embeddings in a new column called `['combined_embedding']` for each form.


In [6]:
 #Combine the text into a single list for Word2Vec
all_texts = [result['name'].tolist()] + result['fieldtype'].tolist() + result['fieldlabel'].tolist()

# Flatten the list of lists to a single list of strings
all_texts = [str(item) for sublist in all_texts for item in sublist]

# Tokenize the text into lists of words
tokenized_texts = [text.split() for text in all_texts]

# Train Word2Vec model
model = Word2Vec(sentences=tokenized_texts, vector_size=100, window=5, min_count=1, workers=4)

# Create Embeddings
def get_embedding(text):
    # If the input is a list, join it to form a single string
    if isinstance(text, list):
        # Convert all elements to string and filter out None or NaN values
        text = ' '.join(str(item) for item in text if pd.notna(item))
    
    # Handle case where text might still be empty after filtering
    if not text:
        return [0] * model.vector_size

    # Tokenize the text
    words = text.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    
    if len(word_vectors) == 0:  # In case no words are in the model
        return [0] * model.vector_size
    
    # Average the word vectors
    return np.mean(word_vectors, axis=0)

def combine_fields(row):
    # Combine the name with all elements from fieldtype and fieldlabel arrays
    fields = [row['name']] + list(row['fieldtype']) + list(row['fieldlabel'])
    
    # Join all values into a single string, ensuring they're all converted to strings
    return ' '.join(str(item) for item in fields if pd.notna(item))

# Apply the function to create a combined text column
result['combined_text'] = result.apply(combine_fields, axis=1)

# Generate embeddings for the combined text
result['combined_embedding'] = result['combined_text'].apply(get_embedding)
# Display the results
display(result[['name', 'fieldtype', 'fieldlabel','combined_embedding']])

,name,fieldtype,fieldlabel,combined_embedding
0,Projektportefølje,"[autonum, created, createdby, relation, string...","[Projekt nr., Oprettet den, Oprettet af, Proje...","[-0.026978955, 0.030760588, 0.0069841384, 0.00..."
1,Projektområde,[string],[Tekst],"[-0.0024110535, -0.0012770921, 0.0050074793, 0..."
2,Claims,"[userrelation, text, relation, text, createdby...","[Processor, Description of claim, Complaint ca...","[-0.027985519, 0.030913027, 0.006309611, 0.000..."
3,Kundereklamation,"[listselect, string, string, string, string, s...","[Vælg kunde, Gadenavn, Postnr, By, Telefonnumm...","[-0.041987948, 0.048192676, 0.009795453, 0.003..."
4,Reklamation,"[listselect, string, string, userrelation, rel...","[Vælg kunde, E-mail, Kontaktperson, Behandles ...","[-0.046166975, 0.05369904, 0.010756338, 0.0028..."
...,...,...,...,...
753,Strategiske initiativer,"[text, relation, string, relation]","[Tekst, MWB, Id, Periode]","[-0.0025060556, 0.00070658623, -0.0013309738, ..."
754,SI - Must win battles,[string],[MWB],"[-0.05942791, 0.06913444, 0.011842064, 0.00108..."
755,SI - perioder,[string],[Periode],"[-0.08395131, 0.09355178, 0.01572402, 0.004300..."
756,SI - Aktiviteter,"[text, date, combo, userrelation, relation, te...","[Aktivitet, Deadline, Status, Superviserende l...","[-0.02919159, 0.034663383, 0.0044938293, 0.002..."


# Model Training

we will focus on training a machine learning model using the embeddings created previously.

## Fitting the Combined Embedding to K-Nearest Neighbors (KNN)

1. **Fit the `combined_embedding`** data to a K-Nearest Neighbors (KNN) model. This model will allow us to identify the similarity between different forms based on their embeddings.

## Returning the Nearest Neighbors

3. Once the model is trained, use it to return the **nearest 5 neighbors** for any given form. This will help in identifying similar forms based on their embeddings.



In [7]:
knn = NearestNeighbors(n_neighbors=5)  # We want the 5 closest neighbor
knn.fit(np.vstack(result['combined_embedding'].values))

NearestNeighbors()

# Testing the Model

we will test the K-Nearest Neighbors (KNN) model using a new form and perform the necessary steps to analyze the results.

## Data Processing for New Form

1. **Input**: Provide the model with a new form consisting of:
   - **Form Name**
   - **3 Fields** (including field types and labels)

2. **Apply the same data processing** steps as done during training.

## Finding the Nearest Neighbors

3. Use the trained KNN model to find the **5 nearest neighbors** of the new form based on the `combined_embedding`.

## Filtering Common Fields

4. From the output, **filter out the common fields** between the new form and its nearest neighbors. This will help us focus on the unique aspects of the new form.

## Printing Uncommon Labels

5. Finally, print the **uncommon labels** from the nearest neighbors to highlight the differences. This will help us understand what makes the new form unique compared to the others.

In [8]:
# Example form to predict
new_name = "Sikkerhed"
fieldtypes = ['createdby', 'created', 'responsible']
labels = ['Oprettet af', 'Oprettelsesdato', 'Ansvarlig']

combined_text = ' '.join([new_name] + fieldtypes + labels)
cembedding =get_embedding(combined_text)
cembedding = cembedding.reshape(1, -1)


distances, indices = knn.kneighbors(cembedding)

closest_points = result.iloc[indices[0]]  

# Collect all field labels from the nearest neighbors
all_field_labels = [result.iloc[index]['fieldlabel'] for index in indices[0]]

# Flatten the list of field labels and normalize to lowercase
flattened_labels = [label.lower() for labels in all_field_labels for label in labels]

# Count the occurrences of each normalized field label
from collections import Counter
label_counts = Counter(flattened_labels)

# Separate common and uncommon labels
common_labels = [label for label, count in label_counts.items() if count > 3]
uncommon_labels = [label for label, count in label_counts.items() if count == 1]

print("Common Field Labels (case-insensitive):")
for label in common_labels:
    print(label)

print('-----------------------------------------------------')
print("\nUncommon Field Labels (case-insensitive):")
for label in uncommon_labels:
    print(label)

Common Field Labels (case-insensitive):
oprettet
oprettet af
ansvarlig
nummer
evt. relateret afvigelse
følgende udføres
deadline
vælg ansvarlig
hvad er udført
udført dato
verificeret dato
kommentar
evt. relateret reklamation
blank linje
oprettelsesdato
audit område
auditdato
ledende auditor
assisterende auditor
deltagere
auditresultat - resumé
send info til brugere
send info til brugergrupper
opfølgning foretaget
kommentarer
reel forbedring?
vedhæft filer
type
relaterede standarder
komplet auditrapport er vedhæftet?
afvigelser
forbedringsforslag
tidspunkt
-----------------------------------------------------

Uncommon Field Labels (case-insensitive):
sagsbehandling 700 kr
totalt
materialer (kr)
antal timer
løn prod. (kr)
tegnestuen timer
løn tegnestuen (kr)
godkendes af
godkendelse
titel
relateret kundereklamation
titel på opgave
relateret til leverandørreklamation
fra afvigelse
følgende udføres:
deadline for udførelse
ansvarlig for udførelse
hvad er konkret udført
tiltag evalueret den

In [9]:

maxPoints = 0;
overalPoints = 0;
negativePoints = 0;

# Grab form names of Validation Set
#print(dataVal)
for row in dataVal.index:
     # Print the entire row
    #print(f"Row {row}:")
    #print(dataVal.loc[row, "name"])  # Use .loc to access the row by its index
    #print(row)
    form_name = dataVal.loc[row, "name"]
    #print(form_name)
    
    cembeddingForm = get_embedding(form_name)
    cembeddingForm = np.array(cembeddingForm)
    cembeddingForm = cembeddingForm.reshape(1, -1)
    
    distances, indices = knn.kneighbors(cembeddingForm)
    
    closest_points = result.iloc[indices[0]]
    
    # Collect all field labels from the nearest neighbors
    all_field_labels = [result.iloc[index]['fieldlabel'] for index in indices[0]]
    
    # Flatten the list of field labels and normalize to lowercase
    flattened_labels = [label.lower() for labels in all_field_labels for label in labels]
    
    # Count the occurrences of each normalized field label
    from collections import Counter
    label_counts = Counter(flattened_labels)
    
    # Separate common and uncommon labels
    predicted_labels = [label for label, count in label_counts.items() if count > 2]

    #print(dataVal.loc[row, "fieldlabel"])
    #print("-----------")
    #print(predicted_labels)
    
    for label in dataVal.loc[row, "fieldlabel"]:
        maxPoints += 1;
        labels = label.split(",")
        #print(label)
        for predLabel in predicted_labels:
            #print(predLabel.split(","))
            #print(label)
            #print(predLabel)
            labelLower = label.lower()
            if(predLabel == labelLower):
                overalPoints += 1;
                print("matches")
                
    for label in predicted_labels:
        negativePoints += 1;
    print(f"Predicted Captures for '{form_name}': {predicted_labels}")


print(maxPoints)
print(overalPoints)
print(negativePoints)
percentage = overalPoints/maxPoints;
precision = overalPoints / (overalPoints + negativePoints)
precision = precision * 100;
percentage = percentage * 100;
print(f"Precision:'{precision}'")
print(f"percentage:'{percentage}'")
#print(testtest)
#print(labeltest)

Predicted Captures for 'Oprettelse af ipw.dk-site': ['kommentar/ forslag til løsning', '-']
Predicted Captures for 'Budgettal': []
Predicted Captures for 'Lead til salg': []
Predicted Captures for 'Normtid pr uge': []
Predicted Captures for 'Reference demo': []
Predicted Captures for 'Opsigelse af lÃ¸sning': []
Predicted Captures for 'Ulykker og nÃ¦rved hÃ¦ndelser': ['- - - - - -']
Predicted Captures for 'Erfaringer & Forbedringsmuligheder': ['ansvarlig']
Predicted Captures for 'Sygemeldinger': []
Predicted Captures for 'SikkerhedshÃ¦ndelse': []
Predicted Captures for 'Potentiel lÃ¦k af persondata': []
Predicted Captures for 'Kontraktstyring': []
Predicted Captures for 'Udstyr': []
Predicted Captures for 'Informationsaktiver': []
Predicted Captures for 'LeverandÃ¸rer / databehandlere': []
Predicted Captures for 'Leverancer': []
Predicted Captures for 'Statement of Applicability SOA': ['oprettet af', 'oprettet', 'bemærkning', 'nummer', 'konklusion', 'evt. relateret intern audit nr.', 'a